# Clasificación multiclase con redes neuronales en Keras
___

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical

In [ ]:
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

## Importar Dataset

In [ ]:
#https://keras.io/datasets/#fashion-mnist-database-of-fashion-articles

fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

## Explorar el dataset

#### Conjunto de entrenamiento

In [ ]:
X_train_full.shape

In [ ]:
y_train_full.shape

#### Conjunto de prueba

In [ ]:
X_test.shape

In [ ]:
y_test.shape

#### Visualizando una instancia

In [ ]:
# Visualizar la imagen en escala de grises
plt.figure()
plt.imshow(X_train_full[0], cmap='gray')
plt.colorbar()
plt.grid(True)
plt.show()

#### Clases

In [ ]:
y_train_full[0]

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
                   "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
class_names[y_train_full[0]]

#### Visualizando las primeras 25 instancias

In [ ]:
plt.figure(figsize=(10,10))
for i in range(0,25):
    plt.subplot(5,5, i+1)
    plt.imshow(X_train_full[i], cmap='gray')
    plt.title(class_names[(y_train_full[i])])
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
plt.show()

## Preprocesamiento

#### Escalamiento de características

In [ ]:
X_train_full[0]

In [ ]:
X_train_full = X_train_full / 255
X_test = X_test / 255

In [ ]:
X_train_full[0]

## Crear conjunto de validación

In [ ]:
# Separar el conjunto de entrenamiento completo en validación y entrenamiento
X_train = X_train_full[:55000]
y_train = y_train_full[:55000]

X_validation = X_train_full[55000:]
y_validation = y_train_full[55000:]

In [ ]:
X_validation.shape

In [ ]:
X_train.shape

# Modelo

#### Crear el modelo y añadir capas

In [ ]:
from tensorflow.keras import layers
from keras import models

model = keras.models.Sequential()
# Convolutional
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# Dense
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model)

In [ ]:
model.layers

In [ ]:
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
X_validation = X_validation.reshape((X_validation.shape[0], 28, 28, 1))

In [ ]:
train_labels = to_categorical(y_train)
validation_labels = to_categorical(y_validation)
test_labels = to_categorical(y_test)

# Configurar el proceso de aprendizaje (Compilar)
Se especifica:
- Loss function
- Optimizer
- Metrics (Opcional, en caso de querer observar métricas extra durante el entrenamiento o evaluación)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

# Entrenamiento y evaluación en datos de validación
Los datos de validación son opcionales, pero si los pasamos como parámetros en el método `fit()`, Keras medirá el loss y las demás métricas que le indiquemos sobre el conjunto de validación al final de cada época.

Se pueden pasar como parámetros los datos de validación con `validation_data`. Si se usa `validation_split`, la fracción de datos que indiquemos será usada como validación (se tomarán los datos del final del conjunto y no se usarán en el entrenamiento). `validation_data` sobrescribe `validation_split`.

- Si el performance es mucho mejor en el conjunto de entrenamiento que en el de validación hay ->

In [ ]:
history = model.fit(X_train,
                    train_labels,
                    epochs=10,
                    batch_size=32,
                    validation_data=(X_validation,validation_labels))

#### History

Objeto regresado por el método `fit()` que contiene:
- Parámetros de entrenamiento
- La lísta de épocas
- Loss y métricas obtenidas al final de cada época

In [ ]:
history.params

In [ ]:
history.epoch

In [ ]:
history.history

#### Curvas de aprendizaje

In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot(figsize=(15, 8))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
#Si quisiéramos seguir entrenando usando nuestro conjunto de validación
history2 = model.fit(X_train,
                     train_labels,
                     epochs=15,
                     batch_size=32,
                     validation_data=(X_validation,validation_labels),
                     initial_epoch= 10)

In [ ]:
#Si quisiéramos ver las curvas de aprendizaje de un segundo entrenamiento
pd.DataFrame(history2.history).plot(figsize=(15, 8))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

# Evaluación en conjunto de prueba

Estimamos el error de generalización.

In [ ]:
loss_and_metrics = model.evaluate(X_test, test_labels)

# Predicción en nuevos datos

In [ ]:
X_new = X_test[:10]

#### Probabilidad estimada por clase

In [ ]:
y_proba = model.predict(X_new).round(2)
y_proba

In [ ]:
# Convertir probabilidades en clases (el índice de la clase con mayor probabilidad)
y_pred = np.argmax(y_proba, axis=1)

print(y_pred)

#### Clase con mayor probabilidad

In [ ]:
np.array(class_names)[y_pred]

In [ ]:
print(y_test[:10])

# Guardar y restaurar el modelo

#### Guardar modelo

In [ ]:
model.save('keras_fashion.keras') #En formato HDF5


#### Restaurar un modelo

In [ ]:
model = keras.models.load_model('keras_fashion.keras')

In [ ]:
X_new = X_test[:5] # pretend these are new instances
y_pred = model.predict(X_new)
y_pred.round(2)

1. Modifica el modelo par alcanzar un accuracy mayor al 92% pero evitando que suceda el sobreajuste.
2. En lugar de la red personalizada, utiliza un arquitectura de red más compleja como ResNet o VGG.